In [32]:
from system import *

s1 = system('x-o-o-',MMA=True)

Initializing system x-o-o-  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...
Constructing effective Lindblau master equation ...

System  x-o-o-  initialized in 97.6 seconds.


In [38]:
var('gamma','DE','De','g','g_f','Omega','v',domain='positive')
var('G_f')#ratio g/g_f
var('V_f')#ratio v/g_f

b = s1.eff_hamiltonian_gs[1,1](gamma_g=0,gamma_f=gamma)
a = copy(b)
#show(symround(b))
b = b.subs(g=g_f*sqrt(G_f))
b = b.subs(v=g_f*sqrt(V_f))

b = b.operands()[1].operands()[0]

b = b.subs(DE=-0.5*I*gamma+DE*gamma)
b = b.subs(De=-0.5*I*gamma+De*gamma)

var('C_fc','g_f','kappa_c','gamma_f',domain='positive')
b = b.subs(kappa_c = g_f^2/(gamma*C_fc)) 
var('C_fb','g_f','kappa_b','gamma_f',domain='positive')
b = b.subs(kappa_b = g_f^2/(gamma*C_fb))


b = b*gamma  #Omega has to be substituted with Omega/gamma
var('G_g',domain='positive')
b = b.subs( gamma = g_f/G_g)


b=MMA_simplify(b,full=True)
b = symround(b,digits=1)
show(b)

symround: Deleted coefficient 2.0e-16
symround: Deleted coefficient 2.0e-16
symround: Deleted coefficient -1.2e-16
symround: Deleted coefficient 2.0e-16


(C_fb^3*C_fc^3*(De - I)*V_f^3*(e^(3*I*phi) + 1)^2 + (-1.5*I*C_fc*G_f + 2.2*De - 2.2*I)*C_fb^2*C_fc^2*G_g^2*V_f^2*e^(3*I*phi) + (-0.50*I*C_fc*G_f + 0.38*De - 0.38*I)*C_fb*C_fc*G_g^4*V_f*e^(3*I*phi) + (-0.031*I*C_fc*G_f + 0.016*De - 0.016*I)*G_g^6*e^(3*I*phi))/(((DE - I)*De - I*DE - 1)*C_fb^3*C_fc^3*V_f^3*(e^(3*I*phi) + 1)^2 + (((-1.5*I*DE - 1.5)*G_f - 1.5*I*De - 1.5)*C_fc + (2.2*DE - 2.2*I)*De - 2.2*I*DE - 2.2)*C_fb^2*C_fc^2*G_g^2*V_f^2*e^(3*I*phi) + (((-0.50*C_fc - 0.50*I*DE - 0.50)*G_f - 0.50*I*De - 0.50)*C_fc + (0.38*DE - 0.38*I)*De - 0.38*I*DE - 0.38)*C_fb*C_fc*G_g^4*V_f*e^(3*I*phi) + (((-0.062*C_fc - 0.031*I*DE - 0.031)*G_f - 0.031*I*De - 0.031)*C_fc + (0.016*DE - 0.016*I)*De - 0.016*I*DE - 0.016)*G_g^6*e^(3*I*phi))